# Full Technical Report: Where's the American Dream?
**_Exploring Intergenerational Economic Mobility by U.S. County_**

Nateé Johnson  
January 2020

### Overview

The goal of the project was to model and visualize Absolute Upward Mobility (AUM) by county as a function of location-specific characteristics. The aim was to determine which features of a county elevated a low-income child’s opportunity to achieve greater economic outcomes than their parents. My approach was to make a definition of what it meant for a county to be a hotbed of opportunity (a.k.a “Mobile”) versus a place where average upward mobility is hampered (a.k.a “Not Mobile”).  Using binary classification models, I looked at “feature importance” to determine which characteristics of a child’s neighborhood impact their economic outcomes.
Using SciKit-learn in python, I found my Random Forest classifier had greater distinguishing power than my logistic regression model. I encountered missing data (missingno library) and class imbalances (~10% of the counties met the criteria to be labeled as “Mobile”). To address this, I experimented with SMOTE (Synthetic Minority Oversampling Technique) - but found that imputing the median yielded comparable results and was more interpretable. I accounted for the class imbalance within the Random Forest classifier using the ‘class_weight’ parameter. I used python’s Folium library to generate a multi-layer choropleth map with pop-up markers and a dynamic tooltip. 
Prior to modeling and visualizing, I spent a significant amount of time studying the datasets and becoming more familiar with the topic area, which was new to me. I used Pandas to clean data from Opportunity Insights and County Health Rankings, and merged them on the FIPScode (after some fun string manipulation techniques to get them in the same format). Opportunity Insights is a massive effort exploring the topic of Intergenerational Economic Mobility. Researchers from Harvard University, Brown University and the Census Bureau, compiled decades of data from tax returns, W-2s, and U.S. census survey results to link parents and children. Looking at parents’ income and their age at childbirth, they traced those children to adulthood and captured their income at a comparable age. To create my map, I had to reformat the dataframe to fit the structure of the county’s GeoJSON file. This was an important step that allowed the information for each county to be dynamically called from the dataframe as a user hovers over the relevant spatial area. Full functionality of each layer is experienced when viewed singularly. (https://nateej1.github.io/AmericanDream_Geo/)

### Imports
Import libraries and write settings here.

#### External Libraries & Packages

In [4]:
# Data manipulation
import pandas as pd
import pickle
import numpy as np
from pandas_profiling import ProfileReport
import os
import sys

# Options for pandas
pd.options.display.max_columns = 50
pd.options.display.max_rows = 30


# Visualizations
import chart_studio.plotly as py
import folium
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objs as go
from plotly.offline import iplot, init_notebook_mode
init_notebook_mode(connected=True)

# Anaylsis & Modeling Packages
from imblearn.over_sampling import SMOTE
from inspect import signature
from scipy.special import expit
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, roc_curve, auc, confusion_matrix, average_precision_score, precision_recall_curve, classification_report
from sklearn import tree 
from IPython.display import Image  



#### Local Scripts

Import functions to produce styled confusion matrix

In [5]:
src_dir = os.path.join(os.getcwd(), '..', '..', 'src')
sys.path.append(src_dir)

from d04_visualisation.styled_confusion import plot_confusion_matrix

## Data Exploration

### Data Sources & Cleaning

The following section defines the two data sources used in this project. The csv files were orginially inspected with Excel/Numbers. Access to the raw data is provided in [data/01_raw](../../data/01_raw) and links to original sources are provided in each section. 

#### Opportunity Insights

**Original Study:** Where is the Land of Opportunity? *The Geography of Intergenerational Mobility in the United States*  
**Authors:** Chetty, Hendren, Kline and Saez (2014)  
**Publication:** Descriptive Statistics by County and Commuting Zone http://www.equality-of-opportunity.org/assets/documents/mobility_geo.pdf  
**Data source**: http://www.equality-of-opportunity.org/data/ 

**Cleaning Summary:** 
- CSV provided has several sheets. Relevant data is in sheet named "Online Data Table 3". There is substantial metadata in top of sheet, and column headers are multi-indexed. 
- The appropriate column names are preserved by transposing the dataframe, resetting the index with the column names, dropping the level 1 index, then restoring the dataframe to a tidy format with another tranpose. 
- By transposing the dataframe, the datatypes reverted to `objects`. Changed the appropriate columns back to `float`.   


In [ ]:
# This brings in the data while maintaing a multi-indexed column information
county_stats = pd.read_excel('../../data/01_raw/Equality_Opportunity/online_data_tables.xls',
                             sheet_name='Online Data Table 3',
                             header=[29, 30])


# Transposing dataframe to remove first level of multi-indexed column headers
county_wo_multidx = county_stats.T.reset_index(level=1, drop=True).T


county_mobility = pd.concat([county_wo_multidx.iloc[:, :5],
                             county_wo_multidx.iloc[:, 5:].astype('float')], axis=1)  # Transposing the dataframe changes the datatypes, so here we restore the appropriate columns to floats

**Additional Processing**

Adding column "rank_diff" as my definition of a "Mobile" county, and making boolean column
0 is False, meaning a county is NOT considered mobile by my definition. 2804 counties
1 is True, meaning a county IS considered mobile. 337 counties

In [ ]:
county_mobility['rank_diff'] = county_mobility['Absolute Upward Mobility'] - 25

county_mobility['Target'] = (county_mobility.rank_diff >= 25).astype('int')

# Padding FIPS code so the formatting will match when it's time to merge with next dataset
county_mobility['County FIPS Code'] = county_mobility['County FIPS Code'].apply(
    lambda x: str(x).zfill(5))  

# Exporting data as pkl into new notebook for modeling
#county_mobility.to_pickle('../../data/02_intermediate/county_mobility_incomeOnly')

In [ ]:
## Using Pandas Profiling library for comprehensive look at data

ProfileReport(county_mobility)

#### County Health Rankings

Data was originally sources from Tableau Public:  
https://public.tableau.com/s/sites/default/files/media/County_Health_Rankings.csv

Files for individual years can be found at the source:  
https://www.countyhealthrankings.org/app/

In [ ]:
# This returns cleaned County Health data
Clean_CountyHealth_raw()

### Data Decisions

insert image of original data set and talk about decision to average over years '97-'02

Given missing values, date ranges, etc. I will proceed using: 




## Analysis/Modeling

### Modeling on Income Data Only

Decision Tree and Logistic Regression

In choosing features to model on, I am conscious of data leakage and auto-correlation (?). 

In [ ]:
county_mobility = pd.read_pickle('../../data/02_intermediate/county_mobility_incomeOnly')
county_health = pd.read_pickle('../../data/02_intermediate/county_measures')

#### Setting up data

In [ ]:
features = [
    'Top 1% Income Share',
    'Interquartile Income Range',
    'Median Parent Income'
]

X = county_mobility.dropna(subset=features, axis=0)[features]
y = county_mobility.dropna(subset=features, axis=0)[['Target']]
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=5)

X_train, X_test , y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 5) 

**SMOTE - Addressing Unbalanced Classes**

In [ ]:
smote = SMOTE(random_state=42, sampling_strategy=1)
X_res, y_res = smote.fit_resample(X_train, y_train) # np.array(y_train).ravel())

#### Decision Tree 1 - (dropped rows w/ missing data)

In [ ]:
clf = DecisionTreeClassifier(criterion='entropy')
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

**Performance Metrics**

In [ ]:
accuracy_score(y_test, y_pred)
average_precision = average_precision_score(y_test, y_pred)
precision, recall, _ = precision_recall_curve(y_test, y_pred)

# In matplotlib < 1.5, plt.fill_between does not have a 'step' argument
step_kwargs = ({'step': 'post'}
               if 'step' in signature(plt.fill_between).parameters
               else {})
plt.step(recall, precision, color='b', alpha=0.2,
         where='post')
plt.fill_between(recall, precision, alpha=0.2, color='b', **step_kwargs)

plt.xlabel('Recall')
plt.ylabel('Precision')
plt.ylim([0.0, 1.05])
plt.xlim([0.0, 1.0])
plt.title('2-class Precision-Recall curve: AP={0:0.2f}'.format(
          average_precision))
plt.show()

In [ ]:
confusion_matrix(y_test, y_pred), average_precision_score(y_test, y_pred)

**Result Summary**

Initial model exploration: 

 - Split data into 70:30 Train:Test
 - Predicted likelihood of county being classified as "Mobile" (0=false, 1=true) based solely on 
     - 'Top 1% Income Share'
     - 'Interquartile Income Range'
     - 'Median Parent Income'
 - Null values were dropped
 - Only specified `criterion='entropy'`
 
Results: 
 - Model accuracy was 0.866 (Accuracy is defined as the ratio of correctly predicted examples by the total examples.)
 - Average Precision Score = 0.23 (The ratio of correct positive predictions to the total predicted positives) (means I'm OVER predicting Mobility)
     - this means I said a lot of counties were mobile when they really aren't

Next steps:
 - Run same data through logistic regression which will report the likelihood of being classified as Mobile based on the combination of features

#### Decision Tree 2 - (w/ SMOTE )

Given unbalanced class, used SMOTE to resample the data. Now using X_res and y_res for fitting. 

In [ ]:
# instantiating and fitting 2nd tree
clf_res = DecisionTreeClassifier(criterion='entropy', max_depth=2)
clf_res.fit(pd.DataFrame(X_res), pd.DataFrame(y_res))
y_res_pred = clf_res.predict(X_test)

**Performance Metrics**

In [ ]:
accuracy_score(y_test, y_res_pred)
average_precision_score(y_test, y_res_pred)
average_precision2 = average_precision_score(y_test, y_res_pred)
precision2, recall2, _ = precision_recall_curve(y_test, y_res_pred)

# In matplotlib < 1.5, plt.fill_between does not have a 'step' argument
step_kwargs = ({'step': 'post'}
               if 'step' in signature(plt.fill_between).parameters
               else {})
plt.step(recall2, precision2, color='b', alpha=0.2,
         where='post')
plt.fill_between(recall2, precision2, alpha=0.2, color='b', **step_kwargs)

plt.xlabel('Recall')
plt.ylabel('Precision')
plt.ylim([0.0, 1.05])
plt.xlim([0.0, 1.0])
plt.title('2-Class Precision-Recall curve: AP={0:0.2f}'.format(
          average_precision2))
plt.show()

**Result Summary**

- Used split data into 70:30 Train:Test (dropped nulls)
 - Did SMOTE on X_train, y_train
 - Only specified `criterion='entropy'`
 
Results: 
 - Model accuracy was 0.767 (Accuracy is defined as the ratio of correctly predicted examples by the total examples.)
 - Average Precision Score = 0.238 (The ratio of correct positive predictions to the total predicted positives)
     - this means I said a lot of counties were mobile when they really aren't

Next steps:
 - Run same data through second logistic regression which will report the likelihood of being classified as Mobile based on the combination of features

#### Logistic Regression 1

In [ ]:
# Using sklearn first (but no p-values)
regr = LogisticRegression(C=1e5, solver='liblinear')

# train the model using the training sets
regr.fit(np.array(X_train), np.array(y_train).ravel())

prediction = regr.predict(X_test)

**Performance Metrics & Plotting**

In [ ]:
accuracy_score(y_test, predict)
average_precision_score(y_test, prediction)

In [ ]:
plt.figure(1, figsize=(4, 3))
plt.clf()
#plt.scatter(np.array(X).ravel(), np.array(y).ravel(), color='black', zorder=20)

loss = np.array(expit(X_test * regr.coef_ + regr.intercept_)).ravel()
plt.scatter(np.array(X_test).ravel(), loss, color='red', linewidth=3)

# ols = linear_model.LinearRegression()
# ols.fit(X, y)
# plt.plot(X_test, ols.coef_ * X_test + ols.intercept_, linewidth=1)
# plt.axhline(.5, color='.5')

plt.ylabel('y')
plt.xlabel('X')
# plt.xticks(range(-5, 10))
# plt.yticks([0, 0.5, 1])
# plt.ylim(-.25, 1.25)
# plt.xlim(-4, 10)
plt.legend(('Logistic Regression Model', 'Linear Regression Model'),
           loc="lower right", fontsize='small')
plt.tight_layout()
plt.show()

In [ ]:
# Variables needed for plotting the distribution: 

# store the coefficients
coef = regr.coef_
interc = regr.intercept_
# create the linear predictor
lin_pred= (X * coef + interc)
# perform the log transformation
mod_income = 1 / (1 + np.exp(-lin_pred))
# sort the numbers to make sure plot looks right
age_ordered, mod_income_ordered = zip(*sorted(zip(age ,mod_income.ravel()),key=lambda x: x[0]))

# Plotting the distribution

fig = plt.figure(figsize=(8,6))
fig.suptitle('logistic regression', fontsize=16)
plt.scatter(age, income_bin)
plt.xlabel("age", fontsize=14)
plt.ylabel("monthly income", fontsize=14)
plt.plot(age_ordered, mod_income_ordered, c = "black")
plt.show()

**Results Summary**

Initial model exploration: 

 - Used same split data from decision tree: 
     - had to make the y_train a 1-d array
     - null values were dropped
 - Predicted likelihood of county being classified as "Mobile" (0=false, 1=true) based solely on 
     - 'Top 1% Income Share'
     - 'Interquartile Income Range'
     - 'Median Parent Income'
 - Only specified `C=1e5, solver='liblinear'`
 
Results: 
 - Model accuracy was 0.886 (Accuracy is defined as the ratio of correctly predicted examples by the total examples.)
 - Average Precision Score = 0.170 (The ratio of correct positive predictions to the total predicted positives)
     - this means I said a lot of counties were mobile when they really aren't

Next steps:
 - do SMOTE to addressed unbalanced classes


#### Logistic Regression 2 - (SMOTE vs "balanced")

In [ ]:
# Using 'balanced' class_weight in the Logistic Regression instance
regr = LogisticRegression(C=1e5, solver='liblinear', class_weight='balanced')

# train the model using the original training sets
regr.fit(np.array(X_train), np.array(y_train).ravel())
prediction = regr.predict(X_test)
accuracy_score(y_test, prediction), average_precision_score(y_test, prediction)

In [ ]:
# Using SMOTE data instead of balanced classes, to compare the performance of SMOTE vs. the module's balancing
regr2 = LogisticRegression(C=1e5, solver='liblinear')

# train the model using the SMOTE-resampled training sets
regr2.fit(np.array(X_res), np.array(y_res).ravel())
y_res_pred2 = regr2.predict(X_test)
accuracy_score(y_test, y_res_pred2), average_precision_score(y_test, y_res_pred2)

**Results Summary**

Using 'balanced' class_weight in the Logistic Regression instance:

`Accuracy Score = 0.771`  
`Precision = 0.248 ` 

Using SMOTE data instead of balanced classes

`Accuracy Score = 0.782  `  
`Precision = 0.257  `

### Modeling on County Income+Health Data merged
Decision Tree, Logistic Regression, Random Forest

In this series an additional performance metric will be included, the Receiver Operator Characteristic curve (ROC curve). The ROC curve illustrates the true positive rate against false positive rate of our classifier.

Recall = the True Positive Rate  (the ratio of the true positive predictions compared to all values that are actually positive)

The ROC curve gives us a graph of the tradeoff between this false positive and true positive rate. The AUC, or area under the curve, gives us a singular metric to compare these. An AUC of 1 being a perfect classifier, and an AUC of .5 being that which has a precision of 50%.

#### Setting up data

In [ ]:
county_imputed = pd.read_pickle('../../data/03_processed/county_merged_imputed')
county_dropped = pd.read_pickle('../../data/03_processed/county_merged_dropped_NaNs')

In [ ]:
county_imputed.columns

In [ ]:
features_total = ['Top 1% Income Share', 'Interquartile Income Range',
                  'Share Between p25 and p75', 'Mean Parent Income', 'Parent Income P25',
                  'Median Parent Income', 'Parent Income P75',
                  'Parent Income P90', 'Parent Income P99',  'Teenage Birth Rate',
                  'Adult obesity', 'Children in poverty',
                  'Daily fine particulate matter', 'Diabetic screening',
                  'Mammography screening', 'Physical inactivity',
                  'Premature Death', 'Preventable hospital stays',
                  'Sexually transmitted infections', 'Unemployment',
                  'Uninsured', 'Violent crime rate']

In [ ]:
X_drop = county_dropped[features_total]
y_drop = county_dropped[['Target']]

X_imp = county_imputed[features_total]
y_imp = county_imputed[['Target']]

X_drop_train, X_drop_test, y_drop_train, y_drop_test = train_test_split(X_drop, y_drop,
                                                                        test_size=0.3,
                                                                        random_state=5)
X_imp_train, X_imp_test, y_imp_train, y_imp_test = train_test_split(X_imp, y_imp,
                                                                    test_size=0.3,
                                                                    random_state=5)

#### Decision Tree 1 - Imputed 

In [ ]:
imp = DecisionTreeClassifier(criterion='entropy')
imp.fit(X_imp_train, y_imp_train)
y_imp_pred = imp.predict(X_imp_test)

**Performance Metrics**

In [ ]:
average_precision_imp = average_precision_score(y_imp_test, y_imp_pred)
accuracy_score(y_imp_test, y_imp_pred), confusion_matrix(y_imp_test, y_imp_pred), average_precision_score(y_imp_test, y_imp_pred)

#### Decision Tree 2 - Dropped

In [ ]:
drop = DecisionTreeClassifier(criterion='entropy')
drop.fit(X_drop_train, y_drop_train)
y_drop_pred = drop.predict(X_drop_test)

**Performance Metrics**

In [ ]:
average_precision_drop = average_precision_score(y_imp_test, y_imp_pred)
accuracy_score(y_drop_test, y_drop_pred), confusion_matrix(y_drop_test, y_drop_pred), average_precision_score(y_drop_test, y_drop_pred)

#### Logistic Regression - Imputed

In [ ]:
# Using 'balanced' class_weight in the Logistic Regression instance
regr_imp = LogisticRegression(
    C=1e5, solver='liblinear', class_weight='balanced')

# train the model using the original training sets
regr_imp.fit(np.array(X_imp_train), np.array(y_imp_train).ravel())
prediction_imp = regr_imp.predict(X_imp_test)

**Performance Metrics**

In [ ]:
accuracy_score(y_imp_test, prediction_imp), average_precision_score(
    y_imp_test, prediction_imp)

In [ ]:
# ROC Curve Plotting
y_imp_score = regr_imp.fit(np.array(X_imp_train),
                             np.array(y_imp_train).ravel()).decision_function(X_imp_test)
fpr_imp, tpr_imp, thresholds_imp = roc_curve(y_imp_test, y_imp_score)
auc(fpr_imp, tpr_imp)

print('AUC: {}'.format(auc(fpr_imp, tpr_imp)))
plt.figure(figsize=(10,8))
lw=2
plt.plot(fpr_imp, tpr_imp, color='darkorange', lw=lw, label='ROC curve')
plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.yticks([i/20.0 for i in range(21)])
plt.xticks([i/20.0 for i in range(21)])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Logistic Regression ROC Curve - "Imputed" Dataset')
plt.show()

In [ ]:
print(classification_report(y_imp_test, prediction_imp, target_names=['Not Mobile', 'Mobile']))

#### Logistic Regression - Dropped

In [ ]:
# Using 'balanced' class_weight in the Logistic Regression instance
regr_drop = LogisticRegression(C=1e5, solver='liblinear', class_weight='balanced')

# train the model using the original training sets
regr_drop.fit(np.array(X_drop_train), np.array(y_drop_train).ravel())
prediction_drop = regr_drop.predict(X_drop_test)

**Performance Metrics**

In [ ]:
accuracy_score(y_drop_test, prediction_drop), average_precision_score(y_drop_test, prediction_drop)

In [ ]:
#Seaborns Beautiful Styling
#sns.set_style("darkgrid", {"axes.facecolor": ".9"})

print('AUC: {}'.format(auc(fpr, tpr)))
plt.figure(figsize=(10,8))
lw = 2
plt.plot(fpr, tpr, color='darkorange',
         lw=lw, label='ROC curve')
plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.yticks([i/20.0 for i in range(21)])
plt.xticks([i/20.0 for i in range(21)])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve - "Dropped" Dataset')
plt.legend(loc="lower right")
plt.show()

#### Random Forest

Only using imputed version of data for this model. 

In [ ]:
randFor = RandomForestClassifier(n_estimators=99, random_state=5)

randFor.fit(X_imp_train, y_imp_train)
rf_pred_train = randFor.predict(X_imp_train)
rf_pred_test = randFor.predict(X_imp_test)

rf_train_score = accuracy_score(y_imp_train, rf_pred_train)
rf_test_score = accuracy_score(y_imp_test, rf_pred_test)
print('Random Forest')
print("Training Accuracy: {:.2} \t\t Testing Accuracy: {:.2}".format(
    rf_train_score*100, rf_test_score*100))
print('Training Confusion Matrix\n', confusion_matrix(y_imp_train, rf_pred_train),
      '\n'+'-'*20+'\n',
      'Test Confusion Matrix\n', confusion_matrix(y_imp_test, rf_pred_test))

In [ ]:
# Using "class_weight" parameter to account for class imbalance
randFor = RandomForestClassifier(n_estimators=99, random_state=5, class_weight='balanced')

randFor.fit(X_imp_train, y_imp_train)
rf_pred_train = randFor.predict(X_imp_train)
rf_pred_test = randFor.predict(X_imp_test)

rf_train_score = accuracy_score(y_imp_train, rf_pred_train)
rf_test_score = accuracy_score(y_imp_test, rf_pred_test)
print('Random Forest')
print("Training Accuracy: {:.2} \t\t Testing Accuracy: {:.2}".format(rf_train_score*100, rf_test_score*100))
print('Training Confusion Matrix\n', confusion_matrix(y_imp_train, rf_pred_train), 
'\n'+'-'*20+'\n', 
'Test Confusion Matrix\n', confusion_matrix(y_imp_test, rf_pred_test))


**Performance Metrics** 

In [ ]:
print(classification_report(y_imp_test, rf_pred_test, target_names=['Not Mobile', 'Mobile']))

In [ ]:
randFor_ROC = randFor.fit(np.array(X_imp_train),
                             np.array(y_imp_train).ravel()).predict_proba(X_imp_test)
fpr_imp, tpr_imp, thresholds_imp = roc_curve(y_imp_test, randFor_ROC[:,1])
auc(fpr_imp, tpr_imp)

In [ ]:
print('AUC: {}'.format(auc(fpr_imp, tpr_imp)))
plt.figure(figsize=(10,8))
lw=2
plt.plot(fpr_imp, tpr_imp, color='darkorange', lw=lw, label='RandomForest ROC curve')
plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.yticks([i/20.0 for i in range(21)])
plt.xticks([i/20.0 for i in range(21)])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('RandomForest ROC Curve - "Imputed" Dataset')
plt.show()

**Feature Importance**

In [ ]:
randFor.feature_importances_

## Results

### Confusion Matrix

In [ ]:
def plot_confusion_matrix(y_true, y_pred, classes,
                          size=(10, 6),
                          normalize=False,
                          title=None,
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    
    Parameters
    ----------
    y_true: list-like
        test/true values
    
    y_pred: list-like
        predicted values
        
    classes: list
        names of classes which predictions can fall into
        
    normalize: bool
        normalize the results (default False, do not normalize)
        
    title: str
        title of confusion matrix
        
    size: tuple (ordered pair)
        height and width in which the figure will display
        
    Returns
    -------
    ax: axis
    """
    if not title:
        if normalize:
            title = 'Normalized confusion matrix'
        else:
            title = 'Confusion matrix, without normalization'

    # Compute confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    
    # Only use the labels that appear in the data
    # classes = classes[unique_labels(y_true, y_pred)]
    #classes = class_names
    
#     if normalize:
#         cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
#         print("Normalized confusion matrix")
#     else:
#         print('Confusion matrix, without normalization')

#     print(cm)

    fig, ax = plt.subplots(figsize=size)
    im = ax.imshow(cm, interpolation='nearest', cmap=cmap)
    ax.figure.colorbar(im, ax=ax)
    # We want to show all ticks...
    ax.set(xticks=np.arange(cm.shape[1]),
           yticks=np.arange(cm.shape[0]),
           # ... and label them with the respective list entries
           xticklabels=classes, yticklabels=classes,
           title=title,
           ylabel='True label',
           xlabel='Predicted label')

    # Rotate the tick labels and set their alignment.
    plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
             rotation_mode="anchor")
    
    # Loop over data dimensions and create text annotations.
    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            ax.text(j, i, format(cm[i, j], fmt),
                    ha="center", va="center",
                    color="white" if cm[i, j] > thresh else "black")
    fig.tight_layout()
    
    return ax

In [ ]:
plot_confusion_matrix(y_imp_train, rf_pred_train, ['Not Mobile', 'Mobile'], title='Random Forest Train Confusion Matrix')
plt.show()
plot_confusion_matrix(y_imp_test, rf_pred_test, ['Not Mobile', 'Mobile'], title='Random Forest Test Confusion Matrix')
plt.show()

### Feature Correlation

In [ ]:
plt.subplots(figsize=(10,6))
sns.scatterplot(x=county_imputed['Mean Parent Income'], y=county_imputed['Mean Child Income'])
plt.title('Parent vs. Child Mean Income')
plt.show()


In [ ]:
sns.pairplot(county_imputed[['Absolute Upward Mobility', 
                             'Target', 'Teenage Birth Rate', 
                             'Unemployment', 'Share Between p25 and p75']])



### Mapping Results

The output from the following code can be found here: http://nateej1.github.io/AmericanDream_Geo

Static Image
![AUM-map](../../AUM-map-preview.png)

In [ ]:
# Instantiating map object
aum_map = folium.Map(location=[37, -95], zoom_start=5)
bins = list(data_imp['Absolute Upward Mobility'].quantile([0, 0.25, 0.5, 0.75, 1]))

choro = folium.Choropleth(
    geo_data=updated_county_json,
    data=data_imp,
    columns=['County FIPS Code', 'Absolute Upward Mobility'],
    key_on='properties.GEO_ID',
    fill_color='YlOrRd',
    fill_opacity=0.5,
    nan_fill_color='gray',
    nan_fill_opacity=0.9,
    line_opacity=0.5,
    legend_name='Absolute Upward Mobility',
    bins=bins,
    reset=True, 
    highlight=True,
    name='Absolute Upward Mobility'
    
)

choro2 = folium.Choropleth(
    geo_data=updated_county_json,
    data=data_imp,
    columns=['County FIPS Code', 'Target'],
    key_on='properties.GEO_ID',
    fill_color='PuBu',
    fill_opacity=0.5,
    nan_fill_color='gray',
    nan_fill_opacity=0.9,
    line_opacity=0.5,
    legend_name='Mobility Outcomes',
    bins=3,
    reset=True, 
    highlight=True,
    name='Child Can Move atleast 1 quartile up'
    )

locations = list(zip(data_imp.lat, data_imp.lon))
popup_content = list(zip(data_imp['County Name'], data_imp['State'], round(data_imp['Absolute Upward Mobility'],1), round(data_imp['Teenage Birth Rate']*100, 1)
                         ))
popups = ['<center> {} County, {} <br>  <b>AUM:</b> {} <br><b>Teen Birth Rate:</b> {}% <br>'.format(
    name, state, aum, share) for (name, state, aum, share) in popup_content]


tooltip = folium.features.GeoJsonTooltip(fields=['NAME', 'State', 'Absolute Upward Mobility', '% Unemployment', '% Teenage Birth Rate', '% Share Between p25 and p75'],
                                         aliases=[
                                             'County', 'State', 'Absolute Upward Mobility', 'Unemployment Rate','Teen Birth Rate', '"Middle Class" Income Share'],
                                         style=('background-color: grey; color: white; font-family:'
                                                'courier new; font-size: 24px; padding: 10px;'),
                                         localize=True)

choro.geojson.add_child(tooltip)

data_imp_copy = data_imp.reset_index()

for idx, row in data_imp_copy.iterrows():
    if row['Absolute Upward Mobility'] >= 60:
        location = locations[idx][0], locations[idx][1]
        marker = folium.Marker(location=location)    
        popup = popups[idx]
        folium.Popup(popup, max_width='150%').add_to(marker)
        icons = folium.Icon(color='green', icon='ok-sign').add_to(marker)
        marker.add_to(choro2)
    elif row['Absolute Upward Mobility'] <= 32:
        location = locations[idx][0], locations[idx][1]
        marker = folium.Marker(location=location)    
        popup = popups[idx]
        folium.Popup(popup, max_width='150%').add_to(marker)
        icons = folium.Icon(color='red').add_to(marker)
        marker.add_to(choro2)
    else: pass

    
choro.add_to(aum_map)
choro2.add_to(aum_map)
folium.LayerControl().add_to(aum_map)
aum_map.save(os.path.join('../../results', 'aum_map8.html'))

## Conclusions and Next Steps

With more robust datasets, this analysis could inform policy decisions that can foster more equitable environments for upward mobility. 

In continuing this investigation, I would like to incorporate education data from Urban Institute, create a map product with more interactive features, allowing the audience to engage in their own exploration of the topic (using Dash)